In [ ]:
import os
import torch
import warnings
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gymnasium as gym
import neptune
from matplotlib import pyplot as plt
from minatar import Environment

In [1]:
!pip install minatar

In [2]:
!pip install torch --upgrade
!pip install triton --upgrade


In [3]:
!pip install neptune-client

PyTorch device setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Seed for reproducible results

In [ ]:
seed = 2025
np.random.seed(seed)
np.random.default_rng(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, input_shape, output_size):
        super(PolicyNet, self).__init__()
        
        c, h, w = input_shape  # Extract channels, height, width

        self.conv1 = nn.Conv2d(c, 16, kernel_size=3, stride=1, padding=1)  # Conv layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1) # Another Conv layer
        self.fc1 = nn.Linear(32 * h * w, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, output_size) # Output layer

    def forward(self, x):
        # Permute the input from (batch_size, height, width, channels) to (batch_size, channels, height, width)
        #need to do this to match shape expected for torch's Conv2d
        x = x.permute(0, 3, 1, 2)  # Reorder dimensions to match PyTorch's Conv2d format: (batch_size, c, h, w)

    #     x = F.relu(self.conv1(x))  # First Conv layer + ReLU
    #     x = F.relu(self.conv2(x))  # Second Conv layer + ReLU
    #     x = torch.flatten(x, start_dim=1)  # Flatten for the FC layer
    #     x = F.relu(self.fc1(x))  # First FC layer
    #     x = self.fc2(x)  # Output logits (no softmax, handled by policy function)
    #     return F.softmax(x, dim=1)  # Action probabilities

In [ ]:
class ReinforceNNLearber:
    def __init__(self, env, device, run, gamma=0.99, learning_rate=9.8e-4, num_episodes = 500):
        self.env = env
        self.run = run  # Store the Neptune run
        self.gamma = gamma #discount factor
        self.learning_rate = learning_rate
        self.device = device
        # Define the policy network

        # Get number of actions from gym action space
        ###THIS CHANGED FOR MINATAR
        n_actions = self.env.num_actions()
        # Reset and get the number of state observations
        ###THIS CHANGED FOR MINATAR
        self.env.reset()
        state=self.env.state()


        ###THIS CHANGED FOR MINATAR
        # Assuming state shape is (10, 10, c), where c is the number of channels

        self.policy_net = PolicyNet(state.shape, n_actions).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)
        # when initialise like this, you're telling the optimizer how large each step (or update) should be in the gradient descent process. In this case, the learning rate (self.learning_rate) controls the size of the step taken during the optimization step.
        self.num_episodes = num_episodes

    def preprocess_state(self, state):
        """
        Preprocesses the state by transforming it to tensor and adding a batch dimension.

        Args:
            state (numpy.ndarray): Input state from the environment.

        Returns:
            torch.Tensor: State tensor with batch dimension.
        """

        #convert into a pytorch tensor. then unsqueeze to add dimension to the tensor at position zero - NN in pytorch expect batched inputs, first dim batch size.
        #.to(self.device) moves the tensor to the device. Because operations on tensors in pytorch must happen on the same device.
        normalized_state = torch.as_tensor(state, dtype=torch.float32, device=self.device)
        normalized_state = normalized_state.unsqueeze(0)

        return normalized_state


    def compute_returns(self, rewards):
        """
        Computes discounted returns.

        Args:
            rewards (list): List of rewards for each time step.

        Returns:
            numpy.ndarray: Array of discounted returns.

        """

        t_steps = np.arange(len(rewards))
        r = rewards * self.gamma ** t_steps # Compute discounted rewards for each time step

        # Compute the discounted cumulative sum in reverse order and then reverse it again to restore the original order.
        r = np.cumsum(r[::-1])[::-1] / self.gamma ** t_steps

        return r
    #What I had before was less efficient but achieved same result:
    #returns = []
    # cumulative_return = 0
    # for _, _, reward, _ in reversed(episode):
    #     cumulative_return = reward + self.gamma * cumulative_return
    #     returns.insert(0, cumulative_return)


    def compute_loss(self,log_probs, returns):
        """
        Computes the policy gradient loss based on the formula.

        Args:
            log_probs (list): List of log probabilities of selected actions.
            returns (numpy.ndarray): Array of discounted returns.

        Returns:
            torch.Tensor: Computed loss.

        """

        policy_loss = []
        for log_prob, returns in zip(log_probs, returns):#pairs each element of the log_prob with the CORRESPONDING element in the returns list. Outputs an iterator of tuples each containing one element from log_prob and one element from returns.
            #zip(epilog_probsode, returns) will create tuples like this: [(log_prob1, return_1),...]
            policy_loss.append(-log_prob * returns)
            # Negative sign for gradient ascent
            #since we want to maximize the expected return but most optimizers minimize a loss function, we accumulate the negative term for each time step
            #for each time step we calculate log_prob*Gt wehich represents how much the chosen action contributed to the overall reward in terms of the log-likelihood of the action multiplied by the expected reward (how much the action contributes to the final rewards, weighted by how likely the policy was to choose that action)

        return torch.stack(policy_loss).sum()
        #we sum all the losses for the entire episode
        #torch.stack(policy_loss) takes the list of tensors stored in policy_loss and combines them into a single tensor, stacking them along a new dimension
        #.sum() is applied to the stacked tensor to sum the individual elements of the loss across the batch


    def train(self):
        self.policy_net.train()  # Ensures the network is in training mode
        episode_rewards = []
        # Main training loop that loops over each episode
        for episode in range(1, self.num_episodes + 1):
            self.env.reset()
            # state() returns a NumPy array of shape (10, 10, c)
            state=self.env.state()

            # Initialize empty lists to store log probabilities and rewards for the episode
            log_probs = []
            episode_reward = []

            # Loop until the episode is done
            while True:
                #print(state.shape)
                state = self.preprocess_state(state) # Preprocesses the state to convert it to tensor
                #print(state.shape)
                #print((state.permute(0, 3, 1, 2)).shape)
                action_probs = self.policy_net(state) # Forward pass through policy network to get action probabilities
                #Passes the current state through the policy network to comput the probabilities of taking each action.
                #returns the action probabilities for the given state as predicted by the policy network. Type: pytorch tensor. Shape: (batch_size, num_actions)

                # Sample an action from the action probabilities
                dist = torch.distributions.Categorical(action_probs)
                #creates a categorical (discrete - since discrete actions) probability distribution based on the action probabilities.
                #The categorical distribution object provides methods to sample actions or comput the log_probabilities of actions
                action = dist.sample()
                #Take the action in the environment

                # Compute the log probability of the sampled action
                log_prob = dist.log_prob(action)
                log_probs.append(log_prob)

                # Take a step in the environment
                reward, done = self.env.act(action.item())
                next_state=self.env.state()
                #same as before except now have .item() which extracts the scalar value from a pytorch tensor.
                # Action is typically a PyTorch tensor representing the action selected by the policy. If action is sampled from a Categorical distribution, its type will be a 0-dimensional tensor (a scalar tensor), for example: tensor(2)
                #.item() extracts the scalar value from the tensor and converts it into a plain python integer which is expected by gym as input to step()
                episode_reward.append(reward)

                state = next_state # Move to the next state

                if done: # if the episode is done
                    returns = self.compute_returns(episode_reward) # Compute the returns (discounted sum of rewards) for the episode
                    loss = self.compute_loss(log_probs, returns)  # Compute the loss for the episode using the log probabilities and returns
                    self.optimizer.zero_grad()
                    #clears the gradients of all parameters (weights and biases) that are being optimized by the optimizer. It ensures that the gradients from the previous backward pass are reset to zero before the new backward pass. Ensures gradients are not accumulated.
                    #In PyTorch, when you perform a forward pass through a neural network and then call .backward() to compute the gradients, the gradients of the parameters in the network are stored in the .grad attribute of each parameter.
                    #These gradients are used for updating the parameters during the optimization step.
                    #By default,  PyTorch accumulates gradients, meaning it adds the gradients computed during each backward pass to the existing ones. This is useful in some cases, such as when you are training with mini-batches and want to accumulate gradients across multiple steps before updating the parameters.
                    #However, for each new training step, you typically want to reset the gradients to zero to avoid carrying over gradients from the previous batch or episode.

                    loss.backward() #computes the gradients of the loss with respect to the parameters of the policy network using backpropagation.
                    self.optimizer.step() #instance of an adam optimizer which updates the policy network's weights based on the gradients that were computed during backpropagation.
                    #Adam optimizer automatically adjusts the learning rate for each parameter based on the gradients and their respective magnitudes.
                    #The learning rate helps to control the step size in parameter updates, ensuring that the network learns at a reasonable pace without overshooting the optimal solution.

                    episode_reward = sum(episode_reward) # Compute the total reward for the episode

                    #print(f"Episode {episode}, Ep Reward: {episode_reward}")
                    self.run["episode_rewards"].log(episode_reward)
                    episode_rewards.append(episode_reward)

                    break # Exit the episode loop

        return episode_rewards

In [ ]:
def main_Minatar(learning_rate,gamma):
    my_api="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjZDg3ZjNlYi04MWI3LTQ1ODctOGIxNS1iNTY3ZjgzMGYzMzYifQ=="
    run = neptune.init_run(
    project="EnergyGridRL/REINFORCE-Minatar",
    api_token=my_api
)  # your credentials
    # Create the environment
    params = {"GAMMA":gamma,#0.99,
               "LR":learning_rate,#9.8e-4
               "num_episodes":10000}
    run["parameters"] = params# Use the grid size in the environment
    env = Environment(env_name="breakout")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    # Create the Q-LEARNING agent

    agent = ReinforceNNLearber(
               env=env,
               device=device,
               run=run,
               gamma=params["GAMMA"],
               learning_rate=params["LR"],
               num_episodes = params["num_episodes"])

    # Train the agent
    episode_rewards = np.array(agent.train())

    # Stop the Neptune run
    run.stop()

    plt.plot(episode_rewards)
    plt.xlabel("Episode")
    plt.ylabel("Total Reward")
    plt.title("Training Performance")
    plt.show()

In [8]:
!pip install swig
!pip install "gymnasium[box2d]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-linux_x86_64.whl size=2379451 sha256=089e93a859883606f22f5ef7a74bf5fb9758190ac10428b13f9ac930f1df4490
  Stored in directory: /root/.cache/pip/wheels/ab/f1/0c/d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


In [10]:
main_Minatar(1e-3,0.99)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/EnergyGridRL/LunarREINFORCE/e/LUN2-6
Using device: cuda
